In [ ]:
from autogluon.multimodal import MultiModalPredictor
import os
from function import *

In [ ]:
path = download_dataset('SkyFusion')

In [ ]:
train_path = os.path.join(path[0], '_annotations.coco.json')
valid_path = os.path.join(path[1], '_annotations.coco.json')
test_path = os.path.join(path[2], '_annotations.coco.json')

In [ ]:
def update_image_paths_in_json(json_path, root_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    if 'images' in data:
        for image in data['images']:
            image['file_name'] = os.path.join(root_dir, image['file_name'])            
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)    
    print(f"수정된 JSON 파일이 저장되었습니다: {json_path}")

update_image_paths_in_json(train_path, path[0])
update_image_paths_in_json(valid_path, path[1])
update_image_paths_in_json(test_path, path[2])

In [ ]:
train_df = coco_to_dataframe(train_path)
test_df = coco_to_dataframe(test_path)

# train_df = train_df[:1000]

In [ ]:
checkpoint_name = "yolox_s"
checkpoint_name = "yolov3_mobilenetv2_8xb24-320-300e_coco"
num_gpus = 1  # only use one GPU

In [ ]:
predictor = MultiModalPredictor(
    hyperparameters={
        "model.mmdet_image.checkpoint_name": checkpoint_name,
        "env.num_gpus": num_gpus,
    },
    problem_type="object_detection",
)

In [ ]:
predictor.fit(
    train_data=train_path,
    tuning_data=valid_path,
    hyperparameters={
    "optimization.max_epochs": 10,  # max number of training epochs, note that we may early stop before this based on validation setting
    "optimization.check_val_every_n_epoch": 3,  # Do 1 validation each 3 epochs
    },
    time_limit=180,
)

In [ ]:
predictor.evaluate(test_path, eval_tool='torchmetrics')